## Tiến Hành thu thập dữ liệu

---

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests as r
import csv
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from datetime import date

#### Dùng requests để lấy data từ trang ```https://www.worldometers.info/coronavirus```

In [2]:
headers = {'User-Agent': 'Mozilla/5.0'}
page = r.get("https://www.worldometers.info/coronavirus", headers)
soup  = BeautifulSoup(page.text, 'html.parser')

#### Hàm lấy tên của cột

In [3]:
def getTitle():
    title = soup.find_all("thead")
    # title
    result_title = []
    result_title.append("Date")
    for t in title:
        result = t.find_all("th")
        for r in result[1:]:
            temp = r.text
            temp = temp.replace("\n", "")
            result_title.append(temp)
        break


    return result_title

#### Hàm lấy dữ liệu từ now, yesterday, 2 days ago

In [4]:
def getData(_type):
    result = []
    # now
    if (_type == 1):
        data = soup.find_all("table", {"id": "main_table_countries_today"})
    # yesterday
    elif (_type == 2):
        data = soup.find_all("table", {"id": "main_table_countries_yesterday"})
    # 2 days ago
    else:
        data = soup.find_all("table", {"id": "main_table_countries_yesterday2"})
    
    for D in data:
        data = D.find_all("tbody")
        for d in data:
            d.find_all("tr")
            for value in d:
                try:
                    temp = value["style"]
                    if (temp != "display: none"):
                        result.append(value)
                except:
                    None
            break    
        break
        
    return result
    

#### Thực hiện các thao tác lấy dữ liệu 

In [5]:
# lay tieu de cua cot
result_title = getTitle()

# lấy thông tin của ngày hôm nay
result_now = getData(1)

# lấy thông tin của ngày hôm qua
result_yesterday = getData(2)

# lấy thông tin của 2 ngày trước
result_twoDaysAgo = getData(3)

In [11]:
with open("Data\Covid.csv", "a", encoding="UTF8") as f_object:
    writer_object = csv.writer(f_object)
    # write data
    for r in result_yesterday:
        temp = []
        temp.append("5/12/2021")
        for i in r.find_all("td")[1:]:
            temp.append(i.text)
        
        #write
        writer_object.writerow(temp)
    f_object.close()

In [12]:
data_covid = pd.read_csv("Data\Covid.csv")

In [13]:
data_covid

,Date,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",...,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,1/12/2021,USA,"49,428,913",NaN,"803,045",NaN,"39,205,902",NaN,"9,419,966","13,201",...,"753,993,369","2,259,195","333,744,223",North America,7,416,0.0,NaN,NaN,"28,225"
1,1/12/2021,India,"34,596,776",NaN,"469,247",NaN,"34,028,506","+10,207","99,023","8,944",...,"642,412,315","459,141","1,399,161,383",Asia,40,"2,982",2.0,NaN,NaN,71
2,1/12/2021,Brazil,"22,094,459",NaN,"614,754",NaN,"21,321,631",NaN,"158,074","8,318",...,"63,776,166","297,057","214,693,373",South America,10,349,3.0,NaN,NaN,736
3,1/12/2021,UK,"10,228,772",NaN,"144,969",NaN,"9,062,561",NaN,"1,021,242",916,...,"362,531,048","5,300,958","68,389,728",Europe,7,472,0.0,NaN,NaN,"14,933"
4,1/12/2021,Russia,"9,636,881",NaN,"275,193",NaN,"8,329,253",NaN,"1,032,435","2,300",...,"224,900,000","1,540,170","146,022,882",Europe,15,531,1.0,NaN,NaN,"7,070"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1563,5/12/2021,Marshall Islands,4,NaN,,NaN,4,NaN,0,NaN,...,NaN,NaN,"59,761",Australia/Oceania,"14,940",NaN,NaN,NaN,NaN,NaN
1564,5/12/2021,Samoa,3,NaN,,NaN,3,NaN,0,NaN,...,NaN,NaN,"200,298",Australia/Oceania,"66,766",NaN,NaN,NaN,NaN,NaN
1565,5/12/2021,Saint Helena,2,NaN,,NaN,2,NaN,0,NaN,...,NaN,NaN,"6,103",Africa,"3,052",NaN,NaN,NaN,NaN,NaN
1566,5/12/2021,Micronesia,1,NaN,,NaN,1,NaN,0,NaN,...,NaN,NaN,"116,745",Australia/Oceania,"116,745",NaN,NaN,NaN,NaN,NaN


In [14]:
sort_df = data_covid.sort_values(by=["Country,Other"])

In [15]:
sort_df

,Date,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",...,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
1222,4/12/2021,Afghanistan,"157,455",+24,"7,312",NaN,"140,723",+103,"9,420","1,124",...,"796,349","19,820","40,178,842",Asia,255,"5,495",50.0,0.6,NaN,234
101,1/12/2021,Afghanistan,"157,289",NaN,"7,308",NaN,"140,549",NaN,"9,432","1,124",...,"792,156","19,723","40,164,434",Asia,255,"5,496",51.0,NaN,NaN,235
774,2/12/2021,Afghanistan,"157,371",+68,"7,309",+1,"140,581",+32,"9,481","1,124",...,"793,293","19,749","40,169,237",Asia,255,"5,496",51.0,2,0.02,236
1446,5/12/2021,Afghanistan,"157,499",+44,"7,316",+4,"140,809",+86,"9,374","1,124",...,"797,783","19,856","40,178,842",Asia,255,"5,492",50.0,1,0.10,233
325,29/11/2021,Afghanistan,"157,261",+33,"7,308",NaN,"140,530",+53,"9,423","1,124",...,"791,514","19,707","40,164,434",Asia,255,"5,496",51.0,0.8,NaN,235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,29/11/2021,Zimbabwe,"134,226",+235,"4,706",+1,"128,703",+48,817,12,...,"1,588,646","104,735","15,168,267",Africa,113,"3,223",10.0,15,0.07,54
776,2/12/2021,Zimbabwe,"135,337",+712,"4,707",NaN,"128,784",+37,"1,846",12,...,"1,599,473","105,440","15,169,450",Africa,112,"3,223",9.0,47,NaN,122
103,1/12/2021,Zimbabwe,"134,625",NaN,"4,707",NaN,"128,747",NaN,"1,171",12,...,"1,593,325","105,043","15,168,267",Africa,113,"3,222",10.0,NaN,NaN,77
552,30/11/2021,Zimbabwe,"134,625",+399,"4,707",+1,"128,747",+44,"1,171",12,...,"1,593,325","105,043","15,168,267",Africa,113,"3,222",10.0,26,0.07,77
